In [19]:
import boto3
import json
import base64
import io
from PIL import Image
from IPython.display import Video
from moviepy.editor import ImageClip, TextClip, CompositeVideoClip

In [20]:
# Initialize AWS SDK to access Bedrock
bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')

# Sample text for summarization
text = """
Get set for a fabulous season of flavour-packed alfresco dining opportunities featuring spring recipes and wine pairings. There’s no better time to lose those winter layers and enjoy lunch in the warming sun, so we’ve come up with some of our favourite spring recipes and wine pairings to help you savour and share a season of deliciousness.

Before we start, let’s look at what fruits and vegetables are in season in spring. In Australia, we’re blessed to access great quality fruit and vegetables all year round, but there’s something very special about creating beautiful meals with in-season produce. Not only is it more economical, but it’s also better for you and the environment. 

 
What fruits and vegetables are typically in season in Spring?

Veggies: Artichoke, asparagus, several varieties of beans, beetroot, broccoli, Brussels sprouts, cabbage, carrot, cauliflower, celery, eggplant, leek, lettuce, mushrooms, onion, parsnip, peas, potato, sweet potato, pumpkin, spinach, tomato, zucchini.

Fruits: Apples, avocados, bananas, blueberries, grapefruits, honeydew, lemon, lime, lychees, mandarins, mangoes, oranges, pineapples, rhubarb, strawberries, watermelon.

Herbs: Basil, chilli, chives, coriander, dill, kaffir lime leaves, mint, oregano, parsley, rosemary, sage, tarragon and thyme.
"""

# Define model ID for Jurassic-2 (from AI21 Labs)
model_id = 'ai21.j2-ultra-v1'

# Call Bedrock API to summarize the text
response = bedrock.invoke_model(
    modelId=model_id,
    body=json.dumps({
        "prompt": f"Summarize the following text: {text}",
        "maxTokens": 100
    }),
    contentType="application/json"
)

# Read the response body from StreamingBody object
response_body = response['body'].read().decode('utf-8')

# Parse the response body into JSON
response_json = json.loads(response_body)

# Extract the summarized text from the response
summary_result = response_json['completions'][0]['data']
summary = summary_result['text']
print("Summary:", summary)


Summary: With the warmer weather of spring comes the opportunity to enjoy al fresco dining with flavor-packed recipes matched perfectly with wine. In this article you'll find information on:
*Ideas for dishes and wines to serve at your spring lunches
*Advice on how to create these spring recipes and wine pairings.

To start your spring off fabulously, there are certain types of produce that are in season in Australia at this time. Here is a general list:


In [23]:
# Define model ID for Stable Diffusion (Stability AI)
model_id = 'stability.stable-diffusion-xl-v1'

# Generate an image based on the summary
response = bedrock.invoke_model(
    modelId=model_id,
    body=json.dumps({
        "text_prompts": [{"text": summary}],
        "cfg_scale": 7,
        "steps": 50,
        "seed": 0
    }),
    contentType="application/json"
)

# Read the response body from StreamingBody object
response_body = response['body'].read().decode('utf-8')

# Parse the response body into JSON
response_json = json.loads(response_body)

# Extract the image data (depending on how Bedrock provides it)
# Usually, it will be some kind of URL or base64 image data
image_data = response_json['artifacts'][0]['base64']

if image_data:
    image_bytes = base64.b64decode(image_data)
    image = Image.open(io.BytesIO(image_bytes))
    image.save("generated_image.png")
    image.show()
else:
    print("No image data in response.")


In [28]:
# Load your image
image_clip = ImageClip("generated_image.png").set_duration(5)  # Ensure the duration is set

# Create a TextClip for text overlay (ensure fontsize and duration are specified)
txt_clip = TextClip(summary, fontsize=24, color='white').set_position('center').set_duration(5)

# Combine the image and text clips
video = CompositeVideoClip([image_clip, txt_clip])

# Save the final video
video.write_videofile("output_video.mp4", fps=24)

Moviepy - Building video output_video.mp4.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4


In [29]:
# Display the generated video
Video("output_video.mp4", embed=True)
